In [ ]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

import numpy as np
import aeon
import torch
from torch import Tensor
import torch.nn as nn
import torch.functional as F
import pandas as pd
from aeon.datasets.tser_datasets import tser_soton; tser_soton = sorted(list(tser_soton))
from aeon.datasets import load_regression
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import RidgeCV
from tqdm import tqdm

from utils.utils import print_name, print_shape
from preprocessing.stream_transforms import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from random_sig_fourier import SigTensorisedRandProj
from signature import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureExtractor, TabularTimeseriesFeatures, RandomGuesser
from randomized_sig import RandomizedSignature
from rocket_wrappers import RocketWrapper, MiniRocketWrapper, MultiRocketWrapper

np.set_printoptions(precision=3, threshold=5) # Print options

In [ ]:
#############################################
#######          Dataset Code         #######
#############################################

def get_aeon_dataset(
        dataset_name:str, 
        #extract_path = "/rds/general/user/nz423/home/Data/TSER/"
        extract_path = "/home/nikita/hdd/Data/TSER/",
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        ):
    """Loads a dataset from the UCR/UEA archive using 
    the aeon library.

    Args:
        dataset_name (str): Name of the dataset

    Returns:
        Tuple: 4-tuple of the form (X_train, y_train, X_test, y_test)
    """
    X_train, y_train = load_regression(dataset_name, split="train", extract_path=extract_path)
    X_test, y_test = load_regression(dataset_name, split="test", extract_path=extract_path)
    X_train = torch.from_numpy(X_train.transpose(0,2,1)).to(device)
    X_test = torch.from_numpy(X_test.transpose(0,2,1)).to(device)
    return X_train, y_train, X_test, y_test

In [ ]:
def test_get_data(idx: int = 17):
    name = tser_soton[idx]
    X_train, y_train, X_test, y_test = get_aeon_dataset(name, device="cpu")
    print("Dataset:", name)
    print("idx:", idx)
    print("X_train", X_train.shape)
    print("X_test", X_test.shape)

# for i in range(20):
#     test_get_data(i)
#     print("\n")

In [ ]:
##################################
####  Linear Model (Ridge)  ######
##################################

def train_and_test_linear(
        train_X, train_y, test_X, test_y,
        feat_extractor: TimeseriesFeatureExtractor,
        apply_augmentation:bool=True,
        normalize_features:bool=True,
        clf=RidgeCV(alphas=np.logspace(-3, 3, 20))
    ):
    # augment data
    print(train_X.shape)
    if apply_augmentation:
        train_X, test_X = normalize_mean_std_traindata(train_X, test_X)
        train_X = add_basepoint_zero(train_X)
        train_X = augment_time(train_X)
        test_X = add_basepoint_zero(test_X)
        test_X = augment_time(test_X)

    # fit transformer
    t0 = time.time()
    feat_extractor.fit(train_X)
    feat_train_X = feat_extractor.transform(train_X).cpu().numpy()
    feat_test_X = feat_extractor.transform(test_X).cpu().numpy()
    print("feat_train_X", feat_train_X.shape)
    if normalize_features:
        feat_train_X, feat_test_X = normalize_mean_std_traindata(feat_train_X, feat_test_X)


    # feed into linear classifier
    t1 = time.time()
    clf.fit(feat_train_X, train_y)
    t2 = time.time()

    # predict
    pred = clf.predict(feat_test_X)
    test_rmse = root_mean_squared_error(test_y, pred)
    train_rmse = root_mean_squared_error(train_y, clf.predict(feat_train_X))
    alpha = clf.alpha_ if hasattr(clf, 'alpha_') else None
    return train_rmse, test_rmse, alpha, t1-t0, t2-t1

In [ ]:
def run_allModels_singleDataset(X_train, y_train, X_test, y_test):
    max_batch = 32
    trunc_level = 4
    n_features = 500

    models = [
        ["Random Guesser", RandomGuesser()],
        ["Tabular", TabularTimeseriesFeatures()],
        # ["Sig", SigTransform(trunc_level, max_batch)],
        # ["Log Sig", LogSigTransform(trunc_level, max_batch)],
        ["Randomized Signature", RandomizedSignature(
            n_features,
            activation = "tanh",
            max_batch=10,
            )],
        ["TRP", SigTensorisedRandProj(
            trunc_level,
            n_features,
            only_last=True,
            method="linear",
            max_batch=max_batch,
            )],
        ["TRP rbf", SigTensorisedRandProj(
            trunc_level,
            n_features,
            only_last=True,
            method="RBF",
            sigma_rbf=1.0,
            max_batch=max_batch,
            )],
        ["concat TRP", SigTensorisedRandProj(
            trunc_level,
            n_features // (trunc_level-1),
            only_last=False,
            method="linear",
            max_batch=max_batch,
            )],
        ["concat TRP rbf", SigTensorisedRandProj(
            trunc_level,
            n_features // (trunc_level-1),
            only_last=False,
            method="RBF",
            sigma_rbf=1.0,
            max_batch=max_batch,
            )],
        ["Rocket", RocketWrapper(
            n_features
            )],
        ["MiniRocket", MiniRocketWrapper(
            n_features
            )],
        ["MultiRocket", MultiRocketWrapper(
            n_features
            )],
        ]

    # Run experiments
    model_names = [name for (name, _) in models]
    results_ridge = []
    for name, model in models:
        print("name", name)
        result = train_and_test_linear(
            X_train, y_train, X_test, y_test, model
            )
        results_ridge.append(result)
        print()
    
    return model_names, results_ridge

In [ ]:
def run_dataset(dataset_name:str):
    X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
    X_train, X_test = normalize_streams(X_train, X_test, max_T=1000)
    y_train, y_test = normalize_mean_std_traindata(y_train, y_test)
    model_names, results_ridge = run_allModels_singleDataset(X_train, y_train, X_test, y_test)
    return model_names, results_ridge

model_names, results_ridge = run_dataset(tser_soton[17])

In [ ]:
def run_allModels_allData(datasets: List[str]):
    #run experiments
    experiments = {}
    failed = {}
    for dataset_name in tqdm(datasets):
        t0 = time.time()
        try:
            print(dataset_name)
            X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)
            X_train, X_test = normalize_streams(X_train, X_test, max_T=1000)
            y_train, y_test = normalize_mean_std_traindata(y_train, y_test)
            N_train = X_train.shape[0]
            N_test = X_test.shape[0]
            T = X_train.shape[1]
            D = X_train.shape[2]
            if N_train<=2000 and D<=20:
                results = run_allModels_singleDataset(
                    X_train, y_train, X_test, y_test
                    )
                experiments[dataset_name] = results
        except Exception as e:
            print(f"Error: {e}")
            failed[dataset_name] = e
        print("Elapsed time", time.time()-t0)
    
    #parse results
    # Define the attributes and methods
    attributes = ["RMSE_train", "RMSE_test", "alpha", "time_transform", "time_fit"]
    
    # Extract model_names from d_res
    model_names = next(iter(experiments.values()))[0]

    # Create and save DataFrames for each attribute and method
    for attribute in attributes:
        df = pd.DataFrame(columns=model_names)
        for dataset_name, (model_names, results_ridge) in experiments.items():
            values = [res[attributes.index(attribute)] for res in results_ridge]
            df.loc[dataset_name] = values

        # Save the DataFrame
        print(df)
        df.to_pickle(f"TESR_{attribute}_results.pkl")

    return experiments, failed

In [ ]:
run_allModels_allData(tser_soton[17:19])

In [ ]:
# Define the attributes and methods
attributes = ["RMSE_train", "RMSE_test", "time_transform", "time_fit", "alpha"]
#data_dir = "https://github.com/nikitazozoulenko/zephyrox/raw/main/Data/TSER/"
data_dir = ""
# Load and store the DataFrames for each attribute and method
dfs = {}
for attribute in attributes:
    filename = f"TESR_{attribute}_results.pkl"
    print(data_dir+filename)
    df = pd.read_pickle(data_dir + filename)
    dfs[attribute] = df

In [ ]:
dfs["RMSE_test"]